# Highway Vehicle Counting Practice Exercise


In this exercise, you will use the YOLO (You Only Look Once) object detection model to analyze a video of highway traffic.
Your task is to count the number of cars that are leaving the highway (coming toward the camera) on the right side and the number of cars that are joining the road on the left side. The video can be found under `Datasets/Example.mp4`.

## Objectives
- Load and process a video using OpenCV.
- Use the YOLO model to detect vehicles in each frame.
- Use OpenCV to manulate the video.
- Track vehicles as they move through the video frames.
- Count the number of vehicles leaving the highway on the right side.
- Count the number of vehicles joining the road on the left side.

## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install opencv-python ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.1/874.1 kB 13.8 MB/s eta 0:00:00


## Load the YOLO Model

You will first need to load the YOLO model. You can use a pre-trained YOLO model for this task.
Write the code to load the YOLO model below:


In [2]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:01<00:00, 3.63MB/s]


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import cv2

video_path = '/content/drive/MyDrive/github/CV/Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [28]:
# Get the width, height, and frames per second of the video
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Height: {height}")
print(f"Width: {width}")
print(f"FPS: {fps}")

Height: 720
Width: 1280
FPS: 25


## Prepare Video Writer to Store the Output

In [29]:
# Set up video codec and output path
output_path = '/content/drive/MyDrive/github/CV/output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes and the counted cars on both sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

## Process Video Frames and Identify Vehicles on the Right and Left Sides

For each frame in the video, use the YOLO model to detect and track vehicles. You'll need to write a loop that processes each frame and applies the YOLO model.
In each frame, after detecting the vehicles, determine whether they are on the left or right side of the highway.
You can use the position of the bounding boxes provided by YOLO to do this.
* The video should display bounding boxes around the detected objects.
* The video should display the confidence along side with the object id and class id of each detected and tracked object.
* The video display the number of vehicles on the left side.
* The video display the number of vehicles on the right side.
* The video should display the line in which you counted the objects that have crossed it and counted.

In [30]:
# Process Video Frames and Identify Vehicles Entering and Exiting
line_position = int(height * 0.5)  # Horizontal line in the middle

# Gap position and size
gap_start = int(width * 0.45)
gap_end = int(width * 0.55)

# Vehicle counters
incoming_count = 0  # Right segment
outgoing_count = 0  # Left segment

# Initialize tracker (using centroid tracking)
from collections import deque

max_history = 50
vehicle_tracks = {}
vehicle_id = 0

# Sets to keep track of counted vehicles
counted_ids_incoming = set()
counted_ids_outgoing = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply the YOLO model on the frame
    results = model(frame)[0]

    detections = []

    for result in results:
        # Get bounding box information
        x1, y1, x2, y2 = result.boxes.xyxy[0].cpu().numpy().astype(int)
        conf = result.boxes.conf[0].cpu().numpy()
        cls = int(result.boxes.cls[0].cpu().numpy())
        label = model.names[cls]

        # Check if the object is a vehicle
        if label in ['car', 'truck', 'bus', 'motorbike']:
            # Calculate the center of the bounding box
            x_center = int((x1 + x2) / 2)
            y_center = int((y1 + y2) / 2)

            detections.append((x1, y1, x2, y2, x_center, y_center))

            # Draw the bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    # Update vehicle tracks
    # Simple centroid tracker
    new_tracks = {}
    for x1, y1, x2, y2, x_center, y_center in detections:
        min_distance = float('inf')
        matched_id = None
        for vid, track in vehicle_tracks.items():
            last_position = track[-1]
            distance = abs(x_center - last_position[0]) + abs(y_center - last_position[1])
            if distance < min_distance and distance < 50:
                min_distance = distance
                matched_id = vid
        if matched_id is not None:
            vehicle_tracks[matched_id].append((x_center, y_center))
            new_tracks[matched_id] = vehicle_tracks[matched_id]
        else:
            vehicle_id += 1
            new_tracks[vehicle_id] = deque(maxlen=max_history)
            new_tracks[vehicle_id].append((x_center, y_center))
    vehicle_tracks = new_tracks

    # Check if any vehicles have crossed the counting line
    for vid, track in vehicle_tracks.items():
        if len(track) >= 2:
            prev_x, prev_y = track[-2]
            curr_x, curr_y = track[-1]

            # Check if vehicle crosses the line outside the gap
            if (prev_y < line_position and curr_y >= line_position) or (prev_y > line_position and curr_y <= line_position):
                if curr_x < gap_start and vid not in counted_ids_outgoing:
                    outgoing_count += 1
                    counted_ids_outgoing.add(vid)
                elif curr_x > gap_end and vid not in counted_ids_incoming:
                    incoming_count += 1
                    counted_ids_incoming.add(vid)

    # Draw the red counting line split with a gap
    # Left segment
    cv2.line(frame, (0, line_position), (gap_start, line_position), (0, 0, 255), 2)
    # Right segment
    cv2.line(frame, (gap_end, line_position), (width, line_position), (0, 0, 255), 2)

    # Display counters on the frame
    cv2.putText(frame, f'Right Side: {incoming_count}', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.putText(frame, f'Left Side: {outgoing_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    # Write the frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 11 cars, 1 bus, 1 train, 1 truck, 143.2ms
Speed: 3.5ms preprocess, 143.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 1 train, 1 truck, 155.8ms
Speed: 4.8ms preprocess, 155.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 1 train, 1 truck, 151.3ms
Speed: 3.8ms preprocess, 151.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 1 train, 2 trucks, 154.5ms
Speed: 2.9ms preprocess, 154.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 1 train, 2 trucks, 157.5ms
Speed: 5.5ms preprocess, 157.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 train, 2 trucks, 143.1ms
Speed: 5.3ms preprocess, 143.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 1 train, 1 truck, 393.8ms
Speed: 4.4ms preprocess, 393.8ms infer